<a href="https://colab.research.google.com/github/Olanle/Project-006-House-price-predictor-full-pipeline-with-deployed-notebook/blob/main/006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Load & Inspect House Price Dataset

import pandas as pd
import numpy as np

df = pd.read_csv("/content/Housing.csv")

In [7]:
print("Rows, Columns:", df.shape)

print("\n--- Head ---")
print(df.head())


print("\n--- Info ---")
df.info()

print("\n--- Numeric Describe ---")
print(df.describe().T)

print("\n--- Missing values ---")
missing = df.isnull().sum().sort_values(ascending=False)
print(missing[missing > 0])

Rows, Columns: (545, 13)

--- Head ---
      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  

--- Info ---
<class 'pandas.core.frame.DataFrame'>
RangeInd

In [12]:
#Check target distribution / summary
if 'price' in df.columns:
    print("\n--- Target (SalePrice) stats ---")
    print(df['price'].describe())
    # quick skew check
    print("Skew:", df['price'].skew())
else:
    print("\nWARNING: 'price' not found. Set the correct target column name.")

#Show categorical columns and a sample of unique values (first 10)
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\nCategorical columns ({len(cat_cols)}): {cat_cols}")
for c in cat_cols:
    print(f" - {c}: {df[c].nunique()} uniques; sample -> {df[c].dropna().unique()[:10]}")


--- Target (SalePrice) stats ---
count    5.450000e+02
mean     4.766729e+06
std      1.870440e+06
min      1.750000e+06
25%      3.430000e+06
50%      4.340000e+06
75%      5.740000e+06
max      1.330000e+07
Name: price, dtype: float64
Skew: 1.2122388370279802

Categorical columns (7): ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']
 - mainroad: 2 uniques; sample -> ['yes' 'no']
 - guestroom: 2 uniques; sample -> ['no' 'yes']
 - basement: 2 uniques; sample -> ['no' 'yes']
 - hotwaterheating: 2 uniques; sample -> ['no' 'yes']
 - airconditioning: 2 uniques; sample -> ['yes' 'no']
 - prefarea: 2 uniques; sample -> ['yes' 'no']
 - furnishingstatus: 3 uniques; sample -> ['furnished' 'semi-furnished' 'unfurnished']


In [10]:
#Define Features, Target & Split Data

from sklearn.model_selection import train_test_split

target = 'price'

#Drop identifier or irrelevant columns
drop_cols = ['Id'] if 'Id' in df.columns else []

#Define features (everything except target + dropped columns)
X = df.drop(columns=[target] + drop_cols)
y = df[target]

#Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training data: {X_train.shape}")
print(f"Testing data:  {X_test.shape}")


Training data: (436, 12)
Testing data:  (109, 12)
